In [1]:
!pip install gradientai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 6.3 MB/s eta 0:00:00


In [2]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = "gJNwa7i3TE7Ln1vMakdmknbS2lTFDQHl"
os.environ['GRADIENT_WORKSPACE_ID'] = "96bfab08-134e-4a85-a115-be7739d4cd5f_workspace"

In [3]:
from gradientai import Gradient